In [ ]:
from odeformerplus.all import *

In [ ]:
reset_seed(42)
DEVICE = get_device()

## train

In [ ]:
ds_train = SymbolicRegressionDataset('dataset/10k/data_all.json')

In [ ]:
dl_train = DataLoader(ds_train, batch_sampler = BucketBatchSampler(ds_train, 64, 100, shuffle=True), collate_fn=collate_fn_trajs)

In [ ]:
model = ODEFormerPlus_vanilla(device=DEVICE)
count_params(model);

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
# sch = get_lr_scheduler(opt, d_model=256, warmup_steps=1000)
sch = None

In [ ]:
def train(epochs, last_step=0, print_every=200, save_every=1000, save_path='model_ckpt/vanilla/'):
    step = last_step
    for epoch in tqdm(range(epochs), desc='Epoch'):
        for batch in tqdm(dl_train, total=len(dl_train), desc='Train', leave=False):
            loss = model.train_step(batch['trajectories'], batch['odes'], opt)
            # sch.step()
            step += 1
            
            if step % print_every == 0:
                print(f'[{epoch}|{step}] loss: {loss:.4e}')
            if step % save_every == 0:
                path = save_path + f'{step}.pt'
                save_ckpt(model, path)
    
    save_ckpt(model, save_path + f'{step}.pt')

In [ ]:
train(20, last_step=12000, save_path='model_ckpt/vanilla/315/')